In [334]:
positive_file = []
negative_file = []

all_file = []

with open("../move_log.log", "r") as f:
    for line in f:
        [predict, probabilities, file_path] = line.split(" ", 2)
        
        target = positive_file if predict == "1" else negative_file
        target.append(
            {
                "file_path": file_path.replace("\n",""),
                "probabilities": float(probabilities),
                "predict": predict
            }
        )
        
all_file = positive_file + negative_file

In [335]:
len(all_file)

31935

In [336]:
len(positive_file)

7405

In [337]:
# negative_file = sorted(negative_file, key=lambda x:x["probabilities"])

In [338]:
def query_file_total_by_keywords(all_file, keywords: list|set, uninclude_keywords: list|set = []):
    # keywords于uninclude_keywords也可以抽象理解成路径，如 "/题库"
    # keywords: 必须全部出现在"file_path"中，才会被提取
    # uninclude_keywords: 只要出现一个，就不会被提取
    
    total = 0
    all_file_path = []
    for row in all_file:
        if all([key_word in row["file_path"] for key_word in keywords]):
            
            if not uninclude_keywords:
                total += 1
                all_file_path.append(row["file_path"])
                continue
                
            
            if not any([key_word in row["file_path"] for key_word in uninclude_keywords]):
                total += 1
                all_file_path.append(row["file_path"])
                
    return total, all_file_path

# 检查"题库"文件夹

# 由于"学前教辅资料"占比比较多，所以优先查看

In [339]:
# 题库中所有文件数量
total, all_file_path = query_file_total_by_keywords(all_file, {"/题库"})
total

8284

In [340]:
# 题库中所有"学前教辅资料"文件数量
specified_total, all_file_path = query_file_total_by_keywords(all_file, {"/题库", "/学前教辅资料"})
specified_total

3843

In [341]:
specified_total / total

0.4639063254466441

In [342]:
# 从题库中筛选出来的所有文件数量
query_file_total_by_keywords(positive_file, {"/题库"})[0]

4065

In [343]:
# 从题库中筛选出来的"学前教辅资料"所有文件(全都是错误的归类)
query_file_total_by_keywords(positive_file, {"/题库", "/学前教辅资料"})[1]

[]

# 从文件名(文件路径)推测

In [344]:
# 从文件路径推测题库文件夹中试卷
specified_total, examination_all_file_path = query_file_total_by_keywords(all_file, {"/题库"}, {"小品", "/学前教辅资料"})
specified_total

4412

In [345]:
# 从文件路径推测题库中筛选出来的文件夹中试卷
positive_specified_total, examination_positive_file_path = query_file_total_by_keywords(positive_file, {"/题库"}, {"小品", "/学前教辅资料"})
positive_specified_total

4063

In [348]:
# 是题库中的文件但是预测结果为False
set(examination_all_file_path) - set(examination_positive_file_path)

{'./data/docx/题库/医学/10T电子书 超全/未分类/004 - 菜根阁整站电子书（123G）/04畅销书2万本（pdf txt doc chm）/1.世界顶级畅销书530本（Word）/中医基础理论（新世纪教材）.doc',
 './data/docx/题库/医学/10T电子书 超全/未分类/世界顶级畅销书530本/一个学中医的女生写出的保养大法.doc',
 './data/docx/题库/医学/10T电子书 超全/👑👑各类分类/书藉大合集/中医资料（1.94T）_分享/南京中医药大学学习课件/按摩手法.doc',
 './data/docx/题库/医学/10T电子书 超全/👑👑各类分类/书藉大合集/中医资料（1.94T）_分享/南京中医药大学学习课件/推拿学基础/2治则.doc',
 './data/docx/题库/医学/10T电子书 超全/👑👑各类分类/书藉大合集/中医资料（1.94T）_分享/南京中医药大学学习课件/推拿学基础/9辅助方法.doc',
 './data/docx/题库/医学/10T电子书 超全/👑👑各类分类/书藉大合集/中医资料（1.94T）_分享/经方系列/傷寒論全部藥方分類錄.doc',
 './data/docx/题库/医学/10T电子书 超全/👑👑各类分类/书藉大合集/中医资料（1.94T）_分享/食疗/生活百科：盘点冬季萝卜食疗效果最好的八种吃法 .doc',
 './data/docx/题库/医学/10T电子书 超全/👑👑各类分类/书藉大合集/中医资料（1.94T）_分享/食疗/茶/冬季养生：吃羊肉五大禁忌 忌与茶同食.doc',
 './data/docx/题库/医学/10T电子书 超全/👑👑各类分类/书藉大合集/中医资料（1.94T）_分享/食疗/茶/生姜红茶 为冬季减肥做准备 .doc',
 './data/docx/题库/医学/10T电子书 超全/👑👑各类分类/书藉大合集/中医资料（1.94T）_分享/食疗/酒/冬季吃火锅喝烈酒悠着点 葡萄酒养生避四大雷区.doc',
 './data/docx/题库/医学/10T电子书 超全/👑👑各类分类/书藉大合集/中医资料（1.94T）_分享/食疗/酒/年底聚会多 男人如何健康饮酒 .doc',
 './data/docx/题库/医学/10T电子书 超全/👑👑各类分类

# 在全部的范围中使用文件名推测

In [349]:
def query_file_total_by_any_keywords(all_file, keywords, uninclude_keywords=[]):
    total = 0
    all_file_path = []
    for row in all_file:
        if any([key_word in row["file_path"] for key_word in keywords]):
            
            if not uninclude_keywords:
                total += 1
                all_file_path.append(row["file_path"])
                continue
                
            
            if not any([key_word in row["file_path"] for key_word in uninclude_keywords]):
                total += 1
                all_file_path.append(row["file_path"])
                
    return total, all_file_path

# 寻找假阴性

In [350]:
# 全部文件中file_name包含关键字的文件
total, all_file_path = query_file_total_by_any_keywords(all_file, {"真题", "试卷", "考试", "答案"}, {"/千套简历模板+简历指导+面试真题-求职加薪必备", "申论", "/培训体系"})
total

4805

In [351]:
# 筛选出来的文件中file_name包含关键字的文件
total, all_classified_file_path = query_file_total_by_any_keywords(positive_file, {"真题", "试卷", "考试", "答案"})
total

4438

In [352]:
set(all_file_path) - set(all_classified_file_path)

{'./data/docx/202307/0003、初中(1)/03、初三/学而思2019初三/2019年初三政治（全套）/【2018-秋】中考思品直播菁英班（人教-部编版）/群资料/中考后五讲讲义及练习题答案/中考思品直播菁英班  第十讲 专题总结和模拟测试.docx',
 './data/docx/202307/01各类考试合集(1)/2022年二级建造师全专业资料/【02】2022年二建【法规】VIP/2/【002】22年二建【法规】VIP课件（推荐）/【04】精讲班（重点学习）/【01】精讲班-二建法规-JG-考点精讲-张子俊/讲义/06-考点精讲（六）.doc',
 './data/docx/202307/01各类考试合集(1)/2022年二级建造师全专业资料/【02】2022年二建【法规】VIP/2/【002】22年二建【法规】VIP课件（推荐）/【04】精讲班（重点学习）/【01】精讲班-二建法规-JG-考点精讲-张子俊/讲义/07-法规-考点精讲（七）.doc',
 './data/docx/202307/01各类考试合集(1)/2022年二级建造师全专业资料/【03】2022年二建【建筑】VIP/2/【003】22年二建【建筑】VIP课件（推荐）/【04】精讲班（重点学习）/【01】精讲班-二建建筑-JG-考点精讲班-杨阳/讲义/04-考点精讲（四）.doc',
 './data/docx/202307/01各类考试合集(1)/2022年二级建造师全专业资料/【03】2022年二建【建筑】VIP/2/【003】22年二建【建筑】VIP课件（推荐）/【04】精讲班（重点学习）/【01】精讲班-二建建筑-JG-考点精讲班-杨阳/讲义/05-考点精讲（五）.doc',
 './data/docx/202307/01各类考试合集(1)/2022年二级建造师全专业资料/【03】2022年二建【建筑】VIP/2/【003】22年二建【建筑】VIP课件（推荐）/【04】精讲班（重点学习）/【01】精讲班-二建建筑-JG-考点精讲班-杨阳/讲义/06-建筑-考点精讲（六）.doc',
 './data/docx/202307/01各类考试合集(1)/2022年二级建造师全专业资料/【03】2022年二建【建筑】VIP/2/【003】22年二建【建筑】VIP课件（推

# 寻找假阳性

In [353]:
# 查找以下关键字都不在路径中存在的
not_required_keywords = {
    "真题", 
    "试卷", 
    "考试", 
    "答案", 
    "高中", 
    "初中",
    "公考遴选", 
    "练习题", 
    "奥数", 
    "单元测试",
    "原卷",
    "习题",
    "笔试题",
    "试题",
    "解析版",
    "/练习",
    "题库"
}

total, all_file_path = query_file_total_by_any_keywords(all_file, {"/"}, not_required_keywords)
total

18387

In [354]:
total, all_classified_file_path = query_file_total_by_any_keywords(negative_file, {"/"}, not_required_keywords)
total

18147

In [358]:
set(all_file_path) - set(all_classified_file_path)

{'./data/docx/202307/10TB精整资料合集【公众号：锦技社】/｜ 精选电子书籍 ｜ @【公众号：锦技社】/_ 精选电子书籍合集 _/掌盟☯️研报中心重磅9月29日更新【内部流出】/01掌盟☯️商业计划书与可行性报告大全/15、物流交通商业计划书/【公益知识库zscc.club】XX物流公司商业计划书.doc',
 './data/docx/202307/10TB精整资料合集【公众号：锦技社】/｜ 精选电子书籍 ｜ @【公众号：锦技社】/_ 精选电子书籍合集 _/掌盟☯️研报中心重磅9月29日更新【内部流出】/01掌盟☯️商业计划书与可行性报告大全/23、化工、能源、环保可行性报告/输变电项目可行性研究报告内容深度/1183023511593/【公益知识库zscc.club】6、GB50217－94电力工程电缆设计规范.doc',
 './data/docx/202307/10TB精整资料合集【公众号：锦技社】/｜ 精选电子书籍 ｜ @【公众号：锦技社】/_ 精选电子书籍合集 _/掌盟☯️研报中心重磅9月29日更新【内部流出】/01掌盟☯️商业计划书与可行性报告大全/32、通讯、广告IT可行性报告/【公益知识库zscc.club】XX网成立的商业计划书.doc',
 './data/docx/202307/10TB精整资料合集【公众号：锦技社】/｜ 精选电子书籍 ｜ @【公众号：锦技社】/_ 精选电子书籍合集 _/掌盟☯️研报中心重磅9月29日更新【内部流出】/01掌盟☯️商业计划书与可行性报告大全/44、项目方案/加工、能源、环保/【公益知识库zscc.club】火力发电厂.doc',
 './data/docx/202307/10TB精整资料合集【公众号：锦技社】/｜ 精选电子书籍 ｜ @【公众号：锦技社】/_ 精选电子书籍合集 _/掌盟☯️研报中心重磅9月29日更新【内部流出】/01掌盟☯️商业计划书与可行性报告大全/44、项目方案/娱乐、旅游服务/【公益知识库zscc.club】旅游资源的市场策略探析.doc',
 './data/docx/202307/10TB精整资料合集【公众号：锦技社】/｜ 精选电子书籍 ｜ @【公众号：锦技社】/_ 精选电子书籍合集 _/掌盟☯️研报中心重磅9月29日更新【内部流出】/01掌盟☯️商业计划书与可行性